In [1]:
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tqdm.auto import tqdm
from matplotlib import gridspec
from statsmodels.graphics.mosaicplot import mosaic

In [2]:
def parse_file_to_dt(fname):
    chrom_states = ["Promoter", "Enhancer", "Transcribed",
                    "PRC_Repressed", "Insulator", "HetChrom_CNV"]
    dt = {k : 1 for k in chrom_states}
    with open(fname, 'r') as fn:
        for line in fn:
            pair = line.strip().split(' ')
            dt[pair[1]] = pair[0]
    return dt

In [23]:
chromHmm_mesc = pd.read_table("./ChromHMM_mm10.mESC.bed", names=["chr", "start", "end", "state"])
chromHmm_mesc["length"] = chromHmm_mesc["end"] - chromHmm_mesc["start"]
statelen = chromHmm_mesc.groupby("state")["length"].sum()
statelen

state
Enhancer          113476400
HetChrom_CNV     2217577800
Insulator          18792000
PRC_Repressed     185155600
Promoter           38233800
Transcribed       152300000
Name: length, dtype: int64

In [32]:
dt = {}

for FILE in glob.glob("./res/*"):
    protein = ".".join(FILE.split('/')[-1].split('.')[:4])
    dt[protein] = parse_file_to_dt(FILE)
        
chip_df = pd.DataFrame.from_dict(dt).transpose()
chip_df = chip_df.astype("float")
#chip_df = chip_df.reset_index().rename(columns={"level_0":"PROTEIN", "level_1":"WINDOW"})
chip_df = chip_df.div(statelen, axis=1) * 10**5#["WINDOW"] = chip_df.WINDOW.apply(lambda x: int(x.split('_')[1]))

chip_df.to_csv("Triads_chromHMM_mouse_densities.tsv", sep='\t')


In [24]:

23402 / 38233800

0.0006120762257478985